This python book will look to predict values of missing values within a pcap before using a model to predict quality resolution. This is compared to the status quo of using the method of dropping values which can be found in our Baseline Netflix IPYNB notebook. 